In [24]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [25]:
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets.cifar import CIFAR10
import torchvision.transforms as T

transforms = T.Compose([
    T.Resize(224),  # VGG는 인풋으로 224*224 크기의 이미지를 받음
    T.RandomCrop((224, 224), padding=4),    # VGG는 인풋으로 224*224 크기의 이미지를 받음
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616))
])

training_data = CIFAR10(
    root="./",
    train=True,
    download=True,
    transform=transforms,
)

test_data = CIFAR10(
    root="./",
    train=False,
    download=True,
    transform=transforms,
)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [30]:
import torch.nn as nn
from torch.optim.adam import Adam
from torchsummary import summary
from torchvision.models.vgg import vgg16

model = vgg16(pretrained=True).to(device)
model.classifier = nn.Sequential(
    nn.Linear(512*7*7, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 10)
).to(device)

lr = 1e-4   # 1e-3으로 하니까 loss가 2.3에서 계속 있었음
optim = Adam(model.parameters(), lr=lr)
cost = nn.CrossEntropyLoss().to(device)

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [31]:
from tqdm import tqdm

epochs = 30

for epoch in range(epochs):
    loss = 0

    with tqdm(train_loader, unit='batch') as loader:
        loader.set_description(f"Epoch {epoch + 1}/{epochs}")

        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            optim.zero_grad()
            preds = model(x)
            loss = cost(preds, y)
            loss.backward()
            optim.step()

            loader.set_postfix(loss=loss.item())

torch.save(model.state_dict(), "CIFAR.pth")

Epoch 7/30:  42%|████▏     | 657/1563 [05:55<08:10,  1.85batch/s, loss=0.0115]


KeyboardInterrupt: ignored

In [22]:
model.load_state_dict(torch.load("CIFAR.pth", map_location=device))

with torch.no_grad():
    x, y = map(lambda x: x.to(device), next(iter(test_loader)))

    output = model(x)
    print(output.max(dim=1))
    print(output.argmax(dim=1))

torch.return_types.max(
values=tensor([1.5302, 0.7816, 0.7794, 1.1564, 1.4565, 0.6136, 0.9488, 1.1170, 1.2109,
        1.5907, 1.4089, 0.8572, 0.6280, 0.7464, 0.4463, 0.2872, 0.3324, 0.2657,
        1.4275, 0.7936, 0.4313, 0.8440, 0.5900, 0.9102, 1.1107, 0.7212, 0.2271,
        1.0129, 0.7293, 0.7362, 0.7978, 0.8983], device='cuda:0'),
indices=tensor([8, 0, 8, 0, 6, 7, 1, 2, 5, 1, 0, 9, 5, 9, 9, 2, 5, 8, 8, 6, 7, 0, 8, 1,
        7, 6, 5, 2, 1, 6, 5, 7], device='cuda:0'))
tensor([8, 0, 8, 0, 6, 7, 1, 2, 5, 1, 0, 9, 5, 9, 9, 2, 5, 8, 8, 6, 7, 0, 8, 1,
        7, 6, 5, 2, 1, 6, 5, 7], device='cuda:0')


In [23]:
model.load_state_dict(torch.load("CIFAR.pth", map_location=device))

num_corr = 0

with torch.no_grad():
    for data, label in test_loader:
        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr

    print(f"Accuracy:{num_corr/len(test_data)}")

Accuracy:0.3657
